In [ ]:
from threat_hunting_toolkit import ThreatHunter
from process_investigation_workbook import ProcessInvestigationWorkbook

hunter = ThreatHunter("byovd_ss.json")
workbook = ProcessInvestigationWorkbook(hunter)

---
# Search by field-value

In [19]:
hunter.search(thread_id = '5204')


[*] Found 70 matching events
[*] Showing first 20 of 70 results
  1. [2025-10-30 03:47:28Z] PID:4 - Thread
  2. [2025-10-30 03:47:33Z] PID:4 - 
  3. [2025-10-30 03:47:33Z] PID:4 - 
  4. [2025-10-30 03:47:33Z] PID:4 - 
  5. [2025-10-30 03:47:33Z] PID:4 - 
  6. [2025-10-30 03:47:33Z] PID:4 - 
  7. [2025-10-30 03:47:33Z] PID:4 - 
  8. [2025-10-30 03:47:33Z] PID:4 - 
  9. [2025-10-30 03:47:33Z] PID:4 - 
  10. [2025-10-30 03:47:33Z] PID:4 - 
  11. [2025-10-30 03:47:33Z] PID:4 - 
  12. [2025-10-30 03:47:33Z] PID:4 - 
  13. [2025-10-30 03:47:33Z] PID:4 - 
  14. [2025-10-30 03:47:33Z] PID:4 - 
  15. [2025-10-30 03:47:33Z] PID:4 - 
  16. [2025-10-30 03:47:33Z] PID:4 - GetFileCache
  17. [2025-10-30 03:47:33Z] PID:4 - GetFileCache
  18. [2025-10-30 03:47:33Z] PID:4 - ValidateImageHeader
  19. [2025-10-30 03:47:33Z] PID:4 - ValidateFileHash
  20. [2025-10-30 03:47:33Z] PID:4 - FileHashFoundInImageCertificate

... and 50 more results not displayed


[{'header': {'activity_id': '{00000000-0000-0000-0000-000000000000}',
   'event_flags': 832,
   'event_id': 0,
   'event_name': '',
   'event_opcode': 3,
   'event_version': 3,
   'process_id': 4,
   'provider_name': 'MSNT_SystemTrace',
   'task_name': 'Thread',
   'thread_id': 5204,
   'timestamp': '2025-10-30 03:47:28Z',
   'trace_name': 'threat_kernel'},
  'properties': {'Affinity': '0x3',
   'BasePriority': 13,
   'IoPriority': 2,
   'PagePriority': 5,
   'ProcessId': 4,
   'StackBase': '0xFFFFE28549A54000',
   'StackLimit': '0xFFFFE28549A4E000',
   'SubProcessTag': 0,
   'TThreadId': 5204,
   'TebBase': '0x0',
   'ThreadFlags': 0,
   'UserStackBase': '0x0',
   'UserStackLimit': '0x0',
   'Win32StartAddr': '0xFFFFF80465E8E4C0'},
  'property_types': {'Affinity': 'POINTER',
   'BasePriority': 'UINT8',
   'IoPriority': 'UINT8',
   'PagePriority': 'UINT8',
   'ProcessId': 'UINT32',
   'StackBase': 'POINTER',
   'StackLimit': 'POINTER',
   'SubProcessTag': 'UINT32',
   'TThreadId': 'UIN

---
# Generic_READ access 

In [18]:
pid_target_pairs = set()
for event in hunter.search('2147483648'): # 2147483648 = GENERIC_READ
    pid, target_pid = event.get('header', {}).get('process_id'), event.get('properties', {}).get('TargetProcessId')
    if pid and target_pid and pid != target_pid:
        pid_target_pairs.add((pid, target_pid))
print(f"\nFound {len(pid_target_pairs)} unique (ProcessID, TargetProcessID) pairs where the source and target are different:\n")
for pair in sorted(list(pid_target_pairs)): print(f"  Source PID: {pair[0]:<8} -> Target PID: {pair[1]}")


[*] Searching for '2147483648' across all fields...
[+] Found 598 events containing '2147483648'

Showing first 20 matches:

  1. [2025-10-30 03:47:28Z] PID:7660
     Found in: properties.DesiredAccess=2147483648

  2. [2025-10-30 03:47:28Z] PID:7660
     Found in: properties.DesiredAccess=2147483648

  3. [2025-10-30 03:47:28Z] PID:7660
     Found in: properties.DesiredAccess=2147483648

  4. [2025-10-30 03:47:28Z] PID:7660
     Found in: properties.DesiredAccess=2147483648

  5. [2025-10-30 03:47:28Z] PID:7660
     Found in: properties.DesiredAccess=2147483648

  6. [2025-10-30 03:47:28Z] PID:7660
     Found in: properties.DesiredAccess=2147483648

  7. [2025-10-30 03:47:28Z] PID:7660
     Found in: properties.DesiredAccess=2147483648

  8. [2025-10-30 03:47:28Z] PID:7660
     Found in: properties.DesiredAccess=2147483648

  9. [2025-10-30 03:47:28Z] PID:7660
     Found in: properties.DesiredAccess=2147483648

  10. [2025-10-30 03:47:28Z] PID:7660
     Found in: properties.DesiredAc

---
# Access Granted to a specific process (using FILTER_PID)

In [40]:
from collections import defaultdict
FILTER_PID = 2648
access_map = defaultdict(set)
for event in hunter.search('GrantedAccess'):
    pid, target_pid, access = event.get('header', {}).get('process_id'), event.get('properties', {}).get('TargetProcessId'), event.get('properties', {}).get('GrantedAccess')
    if pid and target_pid and access and pid != target_pid and (pid == FILTER_PID or target_pid == FILTER_PID): access_map[(pid, target_pid)].add(access)
for (pid, tpid), acc in sorted(access_map.items()): print(f"PID {pid} -> {tpid} | {sorted(list(acc))}")


[*] Searching for 'GrantedAccess' across all fields...
[+] Found 242 events containing 'GrantedAccess'

Showing first 20 matches:

  1. [2025-10-30 03:47:28Z] PID:3856

  2. [2025-10-30 03:47:28Z] PID:3856

  3. [2025-10-30 03:47:28Z] PID:3856

  4. [2025-10-30 03:47:28Z] PID:3856

  5. [2025-10-30 03:47:28Z] PID:3856

  6. [2025-10-30 03:47:28Z] PID:3856

  7. [2025-10-30 03:47:28Z] PID:3856

  8. [2025-10-30 03:47:28Z] PID:3856

  9. [2025-10-30 03:47:28Z] PID:3856

  10. [2025-10-30 03:47:28Z] PID:3856

  11. [2025-10-30 03:47:28Z] PID:3856

  12. [2025-10-30 03:47:28Z] PID:3856

  13. [2025-10-30 03:47:28Z] PID:3856

  14. [2025-10-30 03:47:28Z] PID:3856

  15. [2025-10-30 03:47:28Z] PID:3856

  16. [2025-10-30 03:47:28Z] PID:3856

  17. [2025-10-30 03:47:28Z] PID:3856

  18. [2025-10-30 03:47:33Z] PID:3856

  19. [2025-10-30 03:47:33Z] PID:3856

  20. [2025-10-30 03:47:33Z] PID:3856

... and 222 more results not displayed
💡 Tip: All 242 results are stored in your variable.
      

----
# Search on anything

In [46]:
hunter.search('10140000')


[*] Searching for '10140000' across all fields...
[+] Found 3 events containing '10140000'

Showing all 3 matches:

  1. [2025-10-30 03:47:28Z] PID:4294967295
     Found in: properties.CommitSizeInBytes=0010140000000000

  2. [2025-10-30 03:47:33Z] PID:3856
     Found in: properties.GrantedAccess=10140000

  3. [2025-10-30 03:47:33Z] PID:3856
     Found in: properties.GrantedAccess=10140000


[{'header': {'activity_id': '{00000000-0000-0000-0000-000000000000}',
   'event_flags': 848,
   'event_id': 0,
   'event_name': '',
   'event_opcode': 128,
   'event_version': 2,
   'process_id': 4294967295,
   'provider_name': 'MSNT_SystemTrace',
   'task_name': 'PageFault',
   'thread_id': 4294967295,
   'timestamp': '2025-10-30 03:47:28Z',
   'trace_name': 'virtualalloc_events'},
  'properties': {'BaseAddress': '0x1EF27600000',
   'CommitSizeInBytes': '0010140000000000',
   'Flags': 8192,
   'ProcessId': 6864,
   'RegionSize': '0000200000000000'},
  'property_types': {'BaseAddress': 'POINTER',
   'CommitSizeInBytes': 'OTHER',
   'Flags': 'UINT32',
   'ProcessId': 'UINT32',
   'RegionSize': 'OTHER'}},
 {'header': {'activity_id': '{00000000-0000-0000-0000-000000000000}',
   'event_flags': 576,
   'event_id': 10,
   'event_name': '',
   'event_opcode': 0,
   'event_version': 3,
   'process_id': 3856,
   'provider_name': 'Microsoft-Windows-Sysmon',
   'task_name': 'Process accessed (rul

---
# Accesses Granted to a Process

In [2]:
from collections import defaultdict

# Build PID -> Process Name mapping from MSNT_SystemTrace Process events
pid_to_name = {}
for event in hunter.search(provider_name='MSNT_SystemTrace', task_name='Process'):
    pid = event.get('properties', {}).get('ProcessId')  # 
    image = event.get('properties', {}).get('ImageFileName')
    if pid and image:
        pid_to_name[pid] = image

# Collect GrantedAccess relationships
access_map = defaultdict(set)
for event in hunter.search('GrantedAccess'):
    pid = event.get('properties', {}).get('SourceProcessId')
    target_pid = event.get('properties', {}).get('TargetProcessId')
    access = event.get('properties', {}).get('GrantedAccess')
    if pid and target_pid and access and pid != target_pid:
        access_map[(pid, target_pid)].add(access)

# Display enriched results
print(f"\nFound {len(access_map)} pairs | Resolved {len(pid_to_name)} PIDs\n")
for (pid, target_pid), accesses in sorted(access_map.items()):
    src_name = pid_to_name.get(pid, "UNKNOWN")
    tgt_name = pid_to_name.get(target_pid, "UNKNOWN")
    print(f"  {src_name} ({pid}) -> {tgt_name} ({target_pid}) | {sorted(list(accesses))}")


[*] Found 138 matching events
[*] Showing first 20 of 138 results
  1. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  2. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  3. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  4. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  5. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  6. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  7. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  8. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  9. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  10. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  11. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  12. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  13. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  14. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  15. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  16. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  17. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  18. [2025-10-30 03:47:

---
# Accesses requested by a process

In [61]:
from collections import defaultdict

# Build PID -> Process Name mapping
pid_to_name = {}
for event in hunter.search(provider_name='MSNT_SystemTrace', task_name='Process'):
    pid = event.get('properties', {}).get('ProcessId')
    image = event.get('properties', {}).get('ImageFileName')
    if pid and image:
        pid_to_name[pid] = image

# Collect DesiredAccess relationships
access_map = defaultdict(set)
for event in hunter.search('DesiredAccess'):
    pid = event.get('header', {}).get('process_id')
    target_pid = event.get('properties', {}).get('TargetProcessId')
    access = event.get('properties', {}).get('DesiredAccess')
    if pid and target_pid and access and pid != target_pid:
        access_map[(pid, target_pid)].add(access)

# Display enriched results
print(f"\nFound {len(access_map)} pairs | Resolved {len(pid_to_name)} PIDs\n")
for (pid, target_pid), accesses in sorted(access_map.items()):
    src_name = pid_to_name.get(pid, "UNKNOWN")
    tgt_name = pid_to_name.get(target_pid, "UNKNOWN")
    print(f"  {src_name} ({pid}) -> {tgt_name} ({target_pid}) | {sorted(list(accesses))}")


[*] Found 138 matching events
[*] Showing first 20 of 138 results
  1. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  2. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  3. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  4. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  5. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  6. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  7. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  8. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  9. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  10. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  11. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  12. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  13. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  14. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  15. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  16. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  17. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  18. [2025-10-30 03:47:

---
# Msmpeng.exe is terminated in KernelMode via IOCTL and NOT UserMode

In [70]:
from collections import defaultdict

VICTIM_PID = 2648
VICTIM_NAME = "MsMpEng.exe"

print(f"[+] Hunting for who terminated {VICTIM_NAME} (PID: {VICTIM_PID})")

# Build PID mapping from BOTH Sysmon AND MSNT_SystemTrace
pid_to_name = {}

# Sysmon Event ID 1
for event in hunter.search(provider_name='Microsoft-Windows-Sysmon', event_id=1):
    pid = event.get('properties', {}).get('ProcessId')
    image = event.get('properties', {}).get('Image', '').split('\\')[-1]
    cmdline = event.get('properties', {}).get('CommandLine', '')
    if pid and image:
        pid_to_name[pid] = {'name': image, 'cmdline': cmdline}

# MSNT_SystemTrace (backup for older processes)
for event in hunter.search(provider_name='MSNT_SystemTrace', task_name='Process'):
    pid = event.get('properties', {}).get('ProcessId')
    image = event.get('properties', {}).get('ImageFileName', '')
    if pid and image and pid not in pid_to_name:
        pid_to_name[pid] = {'name': image, 'cmdline': ''}

# Collect access attempts
termination_attempts = []
for event in hunter.search('GrantedAccess'):
    target_pid = event.get('properties', {}).get('TargetProcessId')
    if target_pid == VICTIM_PID:
        src_pid = event.get('properties', {}).get('SourceProcessId')
        access = event.get('properties', {}).get('GrantedAccess')
        timestamp = event.get('header', {}).get('timestamp')
        termination_attempts.append({'pid': src_pid, 'access': access, 'timestamp': timestamp})

for event in hunter.search('DesiredAccess'):
    target_pid = event.get('properties', {}).get('TargetProcessId')
    if target_pid == VICTIM_PID:
        src_pid = event.get('header', {}).get('process_id')
        access = event.get('properties', {}).get('DesiredAccess')
        timestamp = event.get('header', {}).get('timestamp')
        termination_attempts.append({'pid': src_pid, 'access': access, 'timestamp': timestamp})

# Filter for termination rights only (0x0001 bit set)
print(f"\n[!] Processes with TERMINATE rights on {VICTIM_NAME}:\n")
for attempt in sorted(termination_attempts, key=lambda x: x['timestamp']):
    try:
        access_int = int(attempt['access'], 16) if isinstance(attempt['access'], str) else int(attempt['access'])
    except:
        access_int = 0
    
    if access_int & 0x0001:  # Only show termination rights
        killer_info = pid_to_name.get(attempt['pid'], {'name': 'UNKNOWN', 'cmdline': ''})
        flag = "🔴 SELF-TERMINATION" if attempt['pid'] == VICTIM_PID else "⚠️  EXTERNAL TERMINATION"
        print(f"  {attempt['timestamp']} | {killer_info['name']} (PID: {attempt['pid']}) | {hex(access_int)} | {flag}")
        if killer_info['cmdline']:
            print(f"    └─ {killer_info['cmdline'][:100]}")

[+] Hunting for who terminated MsMpEng.exe (PID: 2648)

[*] Found 6 matching events
  1. [2025-10-30 03:47:33Z] PID:3856 - Process Create (rule: ProcessCreate) 
  2. [2025-10-30 03:47:38Z] PID:3856 - Process Create (rule: ProcessCreate) 
  3. [2025-10-30 03:48:09Z] PID:3856 - Process Create (rule: ProcessCreate) 
  4. [2025-10-30 03:48:19Z] PID:3856 - Process Create (rule: ProcessCreate) 
  5. [2025-10-30 03:48:34Z] PID:3856 - Process Create (rule: ProcessCreate) 
  6. [2025-10-30 03:49:07Z] PID:3856 - Process Create (rule: ProcessCreate) 

[*] Found 138 matching events
[*] Showing first 20 of 138 results
  1. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  2. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  3. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  4. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  5. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  6. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  7. [2025-10-30 03:47:28Z] PID:4294967295 - Process
  8. [2025-10-30

In [81]:
# To discover all fields you can use below query. You can also use hunter.lookup_field('field_name'). Below is for GrantedAccess
print("Discovering GrantedAccess event structure...")
sample_events = hunter.search('GrantedAccess')
if sample_events:
    first = sample_events[0] if isinstance(sample_events, list) else next(iter(sample_events), None)
    if first:
        print(f"Provider: {first.get('header', {}).get('provider_name')}")
        print(f"Event ID: {first.get('header', {}).get('event_id')}")
        print(f"Task: {first.get('header', {}).get('task_name')}")
        print(f"Properties keys: {list(first.get('properties', {}).keys())}")

Discovering GrantedAccess event structure...

[*] Searching for 'GrantedAccess' across all fields...
[+] Found 242 events containing 'GrantedAccess'

Showing first 20 matches:

  1. [2025-10-30 03:47:28Z] PID:3856

  2. [2025-10-30 03:47:28Z] PID:3856

  3. [2025-10-30 03:47:28Z] PID:3856

  4. [2025-10-30 03:47:28Z] PID:3856

  5. [2025-10-30 03:47:28Z] PID:3856

  6. [2025-10-30 03:47:28Z] PID:3856

  7. [2025-10-30 03:47:28Z] PID:3856

  8. [2025-10-30 03:47:28Z] PID:3856

  9. [2025-10-30 03:47:28Z] PID:3856

  10. [2025-10-30 03:47:28Z] PID:3856

  11. [2025-10-30 03:47:28Z] PID:3856

  12. [2025-10-30 03:47:28Z] PID:3856

  13. [2025-10-30 03:47:28Z] PID:3856

  14. [2025-10-30 03:47:28Z] PID:3856

  15. [2025-10-30 03:47:28Z] PID:3856

  16. [2025-10-30 03:47:28Z] PID:3856

  17. [2025-10-30 03:47:28Z] PID:3856

  18. [2025-10-30 03:47:33Z] PID:3856

  19. [2025-10-30 03:47:33Z] PID:3856

  20. [2025-10-30 03:47:33Z] PID:3856

... and 222 more results not displayed
💡 Tip: All 24

---
# Process Tree using SystsemTrace + Sysmon (use it on BYOVD_SS.json)

In [20]:
from collections import defaultdict

print("="*70)
print("SYSMON COMPLETE VIEW (Multi-Event Consolidation)")
print("="*70)

# Collect ALL Sysmon events and group by ProcessGuid
process_data = defaultdict(lambda: {
    'events': [],
    'pid': None,
    'parent_guid': None,
    'parent_pid': None,
    'image': None,
    'cmdline': None,
    'user': None,
    'integrity': None,
    'timestamps': [],
    'terminated': False,
    'exit_code': None,
    'children': []
})

# Gather all Sysmon events
for event in hunter.search(provider_name='Microsoft-Windows-Sysmon'):
    props = event.get('properties', {})
    header = event.get('header', {})
    
    # Process as source
    src_guid = props.get('ProcessGuid') or props.get('SourceProcessGUID')
    if src_guid:
        process_data[src_guid]['events'].append(header.get('task_name'))
        process_data[src_guid]['pid'] = process_data[src_guid]['pid'] or props.get('ProcessId')
        process_data[src_guid]['parent_guid'] = process_data[src_guid]['parent_guid'] or props.get('ParentProcessGuid')
        process_data[src_guid]['parent_pid'] = process_data[src_guid]['parent_pid'] or props.get('ParentProcessId')
        process_data[src_guid]['image'] = process_data[src_guid]['image'] or props.get('Image', '').split('\\')[-1]
        process_data[src_guid]['cmdline'] = process_data[src_guid]['cmdline'] or props.get('CommandLine')
        process_data[src_guid]['user'] = process_data[src_guid]['user'] or props.get('User')
        process_data[src_guid]['integrity'] = process_data[src_guid]['integrity'] or props.get('IntegrityLevel')
        
        timestamp = props.get('UtcTime') or header.get('timestamp')
        if timestamp and timestamp not in process_data[src_guid]['timestamps']:
            process_data[src_guid]['timestamps'].append(timestamp)
    
    # Target processes (for terminated events)
    target_guid = props.get('TargetProcessGUID')
    if target_guid:
        process_data[target_guid]['events'].append(f"Target: {header.get('task_name')}")
    
    # Check for termination event (Event ID 5)
    if 'ProcessTerminate' in header.get('task_name', ''):
        term_guid = props.get('ProcessGuid')
        if term_guid:
            process_data[term_guid]['terminated'] = True

# Convert to regular dict with GUIDs as keys
processes = {guid: data for guid, data in process_data.items() if data['image']}

print(f"Collected {len(processes)} unique processes from Sysmon events")

# Link children
for guid, info in processes.items():
    parent_guid = info['parent_guid']
    if parent_guid and parent_guid in processes:
        processes[parent_guid]['children'].append(guid)

# Print tree
def print_tree(guid, indent=0, visited=set()):
    if guid in visited or guid not in processes:
        return
    visited.add(guid)
    info = processes[guid]
    
    term_flag = "💀" if info['terminated'] else ""
    event_count = len(set(info['events']))
    
    print(f"{'  ' * indent}├─ {info['image']} (PID: {info['pid']}, {info['user']}, {info['integrity']}) {term_flag}")
    if info['cmdline']:
        print(f"{'  ' * indent}│  └─ {info['cmdline'][:100]}")
    print(f"{'  ' * indent}│  └─ Events: {event_count} types | Timestamps: {len(info['timestamps'])}")
    
    for child_guid in info['children']:
        print_tree(child_guid, indent + 1, visited)

# Find roots
roots = [g for g, info in processes.items() if info['parent_guid'] not in processes]

print(f"\nProcess Tree ({len(processes)} processes, {len(roots)} roots):")
print("Legend: 💀 = Terminated\n")

for root in sorted(roots, key=lambda g: min(processes[g]['timestamps']) if processes[g]['timestamps'] else ''):
    print_tree(root)

# Statistics
print(f"\n{'='*70}")
print("STATISTICS")
print(f"{'='*70}")
print(f"Total processes:     {len(processes)}")
print(f"Terminated:          {sum(1 for p in processes.values() if p['terminated'])}")
print(f"Avg events/process:  {sum(len(set(p['events'])) for p in processes.values()) / len(processes):.1f}")

SYSMON COMPLETE VIEW (Multi-Event Consolidation)

[*] Found 602 matching events
[*] Showing first 20 of 602 results
  1. [2025-10-30 03:47:28Z] PID:3856 - File created (rule: FileCreate) 
  2. [2025-10-30 03:47:28Z] PID:3856 - File created (rule: FileCreate) 
  3. [2025-10-30 03:47:28Z] PID:3856 - Image loaded (rule: ImageLoad) 
  4. [2025-10-30 03:47:28Z] PID:3856 - Image loaded (rule: ImageLoad) 
  5. [2025-10-30 03:47:28Z] PID:3856 - Image loaded (rule: ImageLoad) 
  6. [2025-10-30 03:47:28Z] PID:3856 - Registry object added or deleted (rule: RegistryEvent) 
  7. [2025-10-30 03:47:28Z] PID:3856 - Registry object added or deleted (rule: RegistryEvent) 
  8. [2025-10-30 03:47:28Z] PID:3856 - Process accessed (rule: ProcessAccess) 
  9. [2025-10-30 03:47:28Z] PID:3856 - Process accessed (rule: ProcessAccess) 
  10. [2025-10-30 03:47:28Z] PID:3856 - Image loaded (rule: ImageLoad) 
  11. [2025-10-30 03:47:28Z] PID:3856 - Image loaded (rule: ImageLoad) 
  12. [2025-10-30 03:47:28Z] PID:38

---
# Event IDs per provider

In [16]:
from collections import defaultdict

event_catalog = defaultdict(lambda: defaultdict(lambda: defaultdict(set)))

for provider, tasks in hunter._event_counts.items():
    for task_name, events in tasks.items():
        for event_name, details in events.items():
            event_ids = details if isinstance(details, set) else {details.get('event_id')}
            event_catalog[provider][task_name][event_name].update(event_ids)

for provider in sorted(event_catalog.keys()):
    print(f"\n{provider}")
    for task_name in sorted(event_catalog[provider].keys()):
        rule = task_name.split('(rule:')[1].split(')')[0].strip() if '(rule:' in task_name else '-'
        for event_name, event_ids in event_catalog[provider][task_name].items():
            ids = sorted([e for e in event_ids if e is not None])
            event_str = f"Event: {event_name}" if event_name != 'NO_EVENT_NAME' else ""
            print(f"  Task: {task_name:<50} Rule: {rule:<25} IDs: {ids} {event_str}")


MSNT_SystemTrace
  Task: DiskIo                                             Rule: -                         IDs: [0] Event: ID:0|Name:NO_EVENT_NAME|Op:34
  Task: DiskIo                                             Rule: -                         IDs: [0] Event: ID:0|Name:NO_EVENT_NAME|Op:52
  Task: DiskIo                                             Rule: -                         IDs: [0] Event: ID:0|Name:NO_EVENT_NAME|Op:37
  Task: DiskIo                                             Rule: -                         IDs: [0] Event: ID:0|Name:NO_EVENT_NAME|Op:53
  Task: DiskIo                                             Rule: -                         IDs: [0] Event: ID:0|Name:NO_EVENT_NAME|Op:35
  Task: Image                                              Rule: -                         IDs: [0] Event: ID:0|Name:NO_EVENT_NAME|Op:3
  Task: Image                                              Rule: -                         IDs: [0] Event: ID:0|Name:NO_EVENT_NAME|Op:10
  Task: Image           